In [ ]:
# MPB version:1.11.1
# Using libctl 4.5.1 and Guile 3.0.9
import math
import meep as mp
from meep import mpb
import numpy as np
import matplotlib.pyplot as plt

a = 0.46  
r = 0.105  
n = 2.805  
epsilon = n**2  

theta1 = 2.205  
theta2 = -2.205  
theta1_rad = math.radians(theta1)
theta2_rad = math.radians(theta2)

geometry_lattice = mp.Lattice(
    basis1=mp.Vector3(math.sqrt(3)/2, 0.5),
    basis2=mp.Vector3(math.sqrt(3)/2, -0.5),
    size=mp.Vector3(5.98, 5.98),  
)

k_points = [
    mp.Vector3(),               # Gamma
    mp.Vector3(-1./3, 1./3),    # K
    mp.Vector3(y=0.5),          # M
    mp.Vector3(),               # Gamma
]

k_points = mp.interpolate(4, k_points)

geometry = [mp.Block(
    size=mp.Vector3(mp.inf, mp.inf, mp.inf),
    material=mp.Medium(epsilon=epsilon)
)]

def generate_triangular_lattice_points(num_cells=13):
    points = []
    for i in range(-num_cells, num_cells+1):
        for j in range(-num_cells, num_cells+1):
            x = i * a + j * (a / 2)
            y = 0 + j * (a * math.sqrt(3) / 2)
            points.append(mp.Vector3(x, y)        )
    return points

lattice_points = generate_triangular_lattice_points()

for p in lattice_points:
    x_rot = p.x * math.cos(theta1_rad) - p.y * math.sin(theta1_rad)
    y_rot = p.x * math.sin(theta1_rad) + p.y * math.cos(theta1_rad)
    geometry.append(mp.Cylinder(
        radius=r,
        center=mp.cartesian_to_lattice(   mp.Vector3(x_rot, y_rot) , geometry_lattice  ), 
        material=mp.Medium(epsilon=1),
        height=mp.inf
    ))

for p in lattice_points:
    x_rot = p.x * math.cos(theta2_rad) - p.y * math.sin(theta2_rad)
    y_rot = p.x * math.sin(theta2_rad) + p.y * math.cos(theta2_rad)
    geometry.append(mp.Cylinder(
        radius=r,
        center=mp.cartesian_to_lattice(    mp.Vector3(x_rot, y_rot)  , geometry_lattice  )   ,
        material=mp.Medium(epsilon=1),
        height=mp.inf
    ))

resolution = 64
num_bands = 300  

ms = mpb.ModeSolver(
    num_bands=num_bands,
    k_points=k_points,
    geometry=geometry,
    geometry_lattice=geometry_lattice,
    resolution=resolution,
)

ms.run_te( )
te_freqs = ms.all_freqs

md = mpb.MPBData(rectify=True, periods=3, resolution=128)
eps = ms.get_epsilon()
converted_eps = md.convert(eps)

plt.figure(figsize=(18, 18))
plt.imshow(converted_eps.T, interpolation='spline36', cmap='binary')
plt.axis('off')
plt.tight_layout()

fig, ax = plt.subplots(figsize=(10, 30))  

x = range(len(te_freqs))
for xz, tez in zip(x, te_freqs):
    ax.scatter([xz]*len(tez), tez, color='blue', s=3, alpha=0.7)

ax.set_ylim([0, 0.7])
ax.set_xlim([x[0], x[-1]])

ax.set_xlabel('k-points', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)

ax.grid(True, alpha=0.3, linestyle='--')

k_point_labels = ['Γ', 'K', 'M', 'Γ']
k_point_positions = [0, len(te_freqs)//3, 2*len(te_freqs)//3, len(te_freqs)-1]

ax.set_xticks(k_point_positions)
ax.set_xticklabels(k_point_labels, fontsize=12)

plt.tight_layout()